In [ ]:
!pip install Sastrawi
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.2 MB/s eta 0:00:00


#Library

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import torch


from transformers import pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
df = pd.read_excel("https://github.com/Adkurrr/digital-residence-identity-sentiment-analysis/blob/main/Dataset/dataset%20dengan%20hasil%20sentimen.xlsx?raw=true")
df

,userName,score,at,content,sentiment
0,Asbak Rokok16batang,5,2024-10-09 13:15:24,Sangat bagus,positive
1,Pengguna Google,1,2024-10-09 11:35:01,Saat verifikasi foto selalu gagal,negative
2,Kiwil Lego,1,2024-10-09 10:34:49,Gak ada guna ny yg berguna tetep kartu ktp ny ...,negative
3,2Dara Network,3,2024-10-09 10:07:42,"Aplikasi belum siap kali, pada auto replay unt...",negative
4,Romli R,1,2024-10-09 09:30:02,Kocak,negative
...,...,...,...,...,...
23873,fredy paska,5,2022-06-09 14:05:40,👍🏻👍🏻👍🏻💪🏻💪🏻💪🏻,positive
23874,Pengguna Google,5,2022-06-09 08:04:34,ga perlu pusing mikirin ktp kapan jadi. sekara...,neutral
23875,Dadan Dharmatin,5,2022-06-09 07:38:42,Muantappss,positive
23876,sultan Czoel,5,2022-06-07 13:46:33,Mantaaap,positive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23878 entries, 0 to 23877
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userName   23878 non-null  object
 1   score      23878 non-null  int64 
 2   at         23878 non-null  object
 3   content    23878 non-null  object
 4   sentiment  23876 non-null  object
dtypes: int64(1), object(4)
memory usage: 932.9+ KB


In [ ]:
df = df.dropna(subset=['sentiment'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23876 entries, 0 to 23877
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userName   23876 non-null  object
 1   score      23876 non-null  int64 
 2   at         23876 non-null  object
 3   content    23876 non-null  object
 4   sentiment  23876 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


In [ ]:
df.duplicated().sum()

0

#Text Preprocessing

###Cleansing (lowercase, hapus emoji, nomor, tanda baca)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean(review):
    review = review.lower()
    review = re.sub('[^a-z A-Z 0-9-]+', '', review)
    review = " ".join([word for word in review.split() if word not in stopwords.words('indonesian')])

    return review

df['content'] = df['content'].apply(clean)
df['content']

,content
0,bagus
1,verifikasi foto gagal
2,gak ny yg berguna tetep kartu ktp ny yg asli n...
3,aplikasi kali auto replay regrestrasi suruh vc...
4,kocak
...,...
23873,
23874,ga pusing mikirin ktp identitas kependudukan d...
23875,muantappss
23876,mantaaap


In [ ]:
df = df[df['content'].str.strip().astype(bool)]
df['content']

,content
0,bagus
1,verifikasi foto gagal
2,gak ny yg berguna tetep kartu ktp ny yg asli n...
3,aplikasi kali auto replay regrestrasi suruh vc...
4,kocak
...,...
23869,kirain aplikasi bikin ktp digital tau y gajelas
23872,aplikasinya bagus membantu wni
23874,ga pusing mikirin ktp identitas kependudukan d...
23875,muantappss


615 baris pada kolom content dihapus karena berisi string kosong

In [ ]:
df.sentiment.value_counts()

,count
sentiment,
negative,10925
positive,8602
neutral,3734


In [ ]:
df.sentiment.value_counts() / len(df.sentiment)

,count
sentiment,
negative,0.469670
positive,0.369804
neutral,0.160526


###Tokenisasi

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df['content'] = df['content'].apply(word_tokenize)
df['content']

,content
0,[bagus]
1,"[verifikasi, foto, gagal]"
2,"[gak, ny, yg, berguna, tetep, kartu, ktp, ny, ..."
3,"[aplikasi, kali, auto, replay, regrestrasi, su..."
4,[kocak]
...,...
23869,"[kirain, aplikasi, bikin, ktp, digital, tau, y..."
23872,"[aplikasinya, bagus, membantu, wni]"
23874,"[ga, pusing, mikirin, ktp, identitas, kependud..."
23875,[muantappss]


###Normalisasi

In [ ]:
url = 'https://github.com/Adkurrr/digital-residence-identity-sentiment-analysis/raw/refs/heads/main/Normalization%20Dataset/normalisasi%20kata%20update.xlsx'
normalisasi_df = pd.read_excel(url, header=None)

# Membuat kamus normalisasi, dengan pengecekan untuk NaN
d = {}
for _, row in normalisasi_df.iterrows():
    k = row[0]
    v = row[1]
    if pd.notna(k) and pd.notna(v):
        d[k.lower()] = v.lower()

# Membuat pola regex untuk pencarian kata yang ingin dinormalisasi
pat = re.compile(r"\b(%s)\b" % "|".join(re.escape(key) for key in d.keys()))


for i in range(len(df)):
    text = df['content'].iloc[i]
    if isinstance(text, list):
        text = ' '.join(text)

    if isinstance(text, str):
        text = text.lower()
        text = pat.sub(lambda m: d[m.group()], text)
        df['content'].iloc[i] = text

print(df['content'].head(10))

Output streaming akan dipotong hingga 5000 baris terakhir.
<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'].iloc[i] = text
<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'].iloc[i] = text
<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'].iloc[i] = text
<ipython-input-32-9530ee2bc57d>:24: SettingWith

0                                                bagus
1                                verifikasi foto gagal
2    tidak nya yang berguna tetep kartu ktp nya yan...
3    aplikasi kali auto replay regrestrasi suruh vc...
4                                                kocak
5                                              barcode
6    tidak berguna buka rek bank bank scan dipermud...
7                                             membantu
8                                             membantu
9                              apk gajelas buang2waktu
Name: content, dtype: object


<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'].iloc[i] = text
<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'].iloc[i] = text
<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'].iloc[i] = text
<ipython-input-32-9530ee2bc57d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

###Filtering
menghilangkan kata yang tidak mempunyai arti


In [ ]:
stop_words = set(stopwords.words('indonesian'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['content'] = df['content'].apply(lambda x: remove_stopwords(x.lower()) if isinstance(x, str) else x)

###Stemming

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return stemmer.stem(text)

df['content'] = df['content'].apply(lambda x: stem_text(x.lower()) if isinstance(x, str) else x)
print(df['content'].head(10))

0                                                bagus
1                                verifikasi foto gagal
2    nya guna tetep kartu ktp nya asli nyetak ktp a...
3    aplikasi kali auto replay regrestrasi suruh vc...
4                                                kocak
5                                              barcode
6    guna buka rek bank bank scan mudah taiii fisik...
7                                                bantu
8                                                bantu
9                              apk gajelas buang2waktu
Name: content, dtype: object


In [ ]:
# df.to_excel('hasil_sentimen.xlsx', index=False)

#Term Weighting TF-IDF

In [ ]:
# Menghitung Term Frequency (TF) Biner: 1 jika ada, 0 jika tidak ada
tf_vectorizer = TfidfVectorizer(use_idf=False, binary=True)
tf = tf_vectorizer.fit_transform(df['content'])

# Menghitung Inverse Document Frequency (IDF)
idf_vectorizer = TfidfVectorizer(use_idf=True)
idf = idf_vectorizer.fit_transform(df['content'])

# Menampilkan hasil TF dan IDF
print("Term Frequency (TF) Biner:")
tf_df = pd.DataFrame(tf.toarray(), columns=tf_vectorizer.get_feature_names_out())
print(tf_df)

idf_values = idf_vectorizer.idf_
idf_df = pd.DataFrame([idf_values], columns=idf_vectorizer.get_feature_names_out())
print("\nInverse Document Frequency (IDF):")
print(idf_df)

# Menghitung dan menampilkan TF-IDF
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf = tfidf_vectorizer.fit_transform(df['content'])
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print("\nTF-IDF:")
print(tfidf_df)

Term Frequency (TF) Biner:
       0000000000100   02   03   04  05102024   06   08  0930   10  100  ...  \
0                0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
1                0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
2                0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
3                0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
4                0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
...              ...  ...  ...  ...       ...  ...  ...   ...  ...  ...  ...   
23256            0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
23257            0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
23258            0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
23259            0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   
23260            0.0  0.0  0.0  0.0       0.0  0.0  0.0   0.0  0.0  0.0  ...   

       zonk 

#Build Model SVM

In [ ]:
X = tfidf
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Akurasi: {accuracy:.2f}")
print("Laporan Klasifikasi:")
print(report)


Akurasi: 0.81
Laporan Klasifikasi:
              precision    recall  f1-score   support

    negative       0.78      0.92      0.84      2199
     neutral       0.65      0.37      0.48       737
    positive       0.89      0.85      0.87      1717

    accuracy                           0.81      4653
   macro avg       0.77      0.71      0.73      4653
weighted avg       0.80      0.81      0.79      4653



Akurasi : 81%

rata2 presisi : 77%

rata2 recall : 71%

rata2 f1-score : 73%

-----------------------------

Penelitian Rita dkk (2023) akurasi sebesar 77%, rata-rata untuk nilai presisi 92%, rata-rata nilai recall 37%, dan rata-rata nilai f1-score 35%.


In [ ]:
prediksi_baru = model_svm.predict(tfidf_vectorizer.transform(['saya sangat senang']))